In [13]:
import pandas as pd
import numpy as np

In [14]:
def maze_summary(dataframe, category):
    '''
    Calculate the means and standard deviations of certain categories in the dataframe.
    dataframe: data table, pandas dataframe
    category: user specified categories for calculation, python list
    '''
    maze_list = dataframe['maze'].unique().tolist()
    mode_list = dataframe['mode'].unique().tolist()
    intuition_list = dataframe['intuition'].unique().tolist()
    summary_mean = pd.DataFrame()
    summary_std = pd.DataFrame()
    for maze in maze_list:
        for mode in mode_list:
            for intuition in intuition_list:
                data = dataframe[(dataframe['maze'] == maze) & (dataframe['mode'] == mode) & (dataframe['intuition'] == intuition)][category]
                data_mean = np.around(data.mean().values, decimals = 2).tolist()
                data_std = np.around(data.std().values, decimals = 2).tolist()
                summary_mean = summary_mean.append([[maze, mode, intuition] + data_mean], ignore_index = True)
                summary_std = summary_std.append([[maze, mode, intuition] + data_std], ignore_index = True)
    return (summary_mean, summary_std)

In [15]:
def file_process(file_list):
    '''
    Process files for maze_summary.
    file_list: file name list, python list
    '''
    summary_mean = pd.DataFrame()
    summary_std = pd.DataFrame()
    for file in file_list:
        dataframe = pd.read_csv(file, sep = ',')
        category = dataframe.columns.values.tolist()[3:]
        summary = maze_summary(dataframe = dataframe, category = category)
        summary_mean = summary_mean.append(summary[0], ignore_index = True)
        summary_std = summary_std.append(summary[1], ignore_index = True)
    return (summary_mean, summary_std)

In [16]:
def summary_process(dataframe_list):
    '''
    Merge statistics of multiple dataframe in a list.
    dataframe_list: dataframe list, python list
    '''
    summary_mean = pd.DataFrame()
    summary_std = pd.DataFrame()
    for dataframe in dataframe_list:
        category = dataframe.columns.values.tolist()[3:]
        summary = maze_summary(dataframe = dataframe, category = category)
        summary_mean = summary_mean.append(summary[0], ignore_index = True)
        summary_std = summary_std.append(summary[1], ignore_index = True)
    return (summary_mean, summary_std)

In [17]:
file_list = ['test_result_maze_01.csv', 'test_result_maze_02.csv', 'test_result_maze_03.csv']

In [18]:
dataframe_list = list()
for file in file_list:
    dataframe_list.append(pd.read_csv(file, sep = ','))

In [19]:
# Take a look at the data
dataframe_list[0].head()

,maze,mode,intuition,num_actions_1,length_movement_1,num_actions_2,length_movement_2,true_coverage,score,computation_time
0,1,complete,False,217.0,280.0,17.0,32.0,1.0,24.800000,89.962
1,1,complete,False,211.0,246.0,17.0,32.0,1.0,24.600000,92.997
2,1,complete,False,194.0,232.0,17.0,32.0,1.0,24.033333,101.822
3,1,complete,False,193.0,234.0,17.0,32.0,1.0,24.000000,89.352
4,1,complete,False,195.0,240.0,17.0,32.0,1.0,24.066667,100.969


In [20]:
# Add maze_size information
# Calculate exploration efficiency
maze_size_dict = {1: 144, 2: 196, 3: 256}
for i in xrange(len(dataframe_list)):
    dataframe_list[i]['maze_size'] = dataframe_list[i]['maze'].map(lambda x: maze_size_dict[x])
    dataframe_list[i]['num_grid_per_action_1'] = dataframe_list[i]['maze_size'] * dataframe_list[i]['true_coverage'] / dataframe_list[i]['num_actions_1']
    dataframe_list[i]['num_grid_per_length_1'] = dataframe_list[i]['maze_size'] * dataframe_list[i]['true_coverage'] / dataframe_list[i]['length_movement_1']

In [21]:
dataframe_list[0].head()

,maze,mode,intuition,num_actions_1,length_movement_1,num_actions_2,length_movement_2,true_coverage,score,computation_time,maze_size,num_grid_per_action_1,num_grid_per_length_1
0,1,complete,False,217.0,280.0,17.0,32.0,1.0,24.800000,89.962,144,0.663594,0.514286
1,1,complete,False,211.0,246.0,17.0,32.0,1.0,24.600000,92.997,144,0.682464,0.585366
2,1,complete,False,194.0,232.0,17.0,32.0,1.0,24.033333,101.822,144,0.742268,0.620690
3,1,complete,False,193.0,234.0,17.0,32.0,1.0,24.000000,89.352,144,0.746114,0.615385
4,1,complete,False,195.0,240.0,17.0,32.0,1.0,24.066667,100.969,144,0.738462,0.600000


In [22]:
column_names = ['Maze', 'Mode', 'Intuition', 'Time Steps in First Run', 'Path Length in First Run', 
                'Time Steps in Second Run', 'Path Length in Second Run', 'Coverage', 'Score', 'Computation Time', 'Maze Size', 'Number of Grid Visited Per Time Step of First Run', 'Number of Grid Visited Per Path Length of First Run']

In [23]:
summary = summary_process(dataframe_list)
summary_mean = summary[0]
summary_std = summary[1]

In [24]:
summary_mean.columns = column_names
summary_std.columns = column_names

In [25]:
summary_mean

,Maze,Mode,Intuition,Time Steps in First Run,Path Length in First Run,Time Steps in Second Run,Path Length in Second Run,Coverage,Score,Computation Time,Maze Size,Number of Grid Visited Per Time Step of First Run,Number of Grid Visited Per Path Length of First Run
0,1,complete,False,200.80,245.0,17.00,32.0,1.00,24.26,92.07,144.0,0.72,0.59
1,1,complete,True,200.90,244.6,17.00,32.0,1.00,24.26,44.49,144.0,0.72,0.59
2,1,incomplete,False,117.15,141.5,18.05,32.2,0.61,22.56,41.78,144.0,0.75,0.62
3,1,incomplete,True,158.75,193.1,17.65,32.1,0.80,23.53,32.74,144.0,0.73,0.61
4,2,complete,False,273.30,332.9,22.00,43.0,1.00,31.84,171.39,196.0,0.72,0.59
5,2,complete,True,280.35,343.9,22.00,43.0,1.00,32.08,82.38,196.0,0.70,0.57
6,2,incomplete,False,195.90,241.7,22.60,43.1,0.73,29.88,95.89,196.0,0.73,0.59
7,2,incomplete,True,188.95,232.3,23.50,43.3,0.70,30.58,47.96,196.0,0.73,0.59
8,3,complete,False,353.95,443.0,25.00,49.0,1.00,37.63,286.99,256.0,0.72,0.58
9,3,complete,True,359.35,450.4,25.00,49.0,1.00,37.81,134.39,256.0,0.71,0.57


In [26]:
summary_std

,Maze,Mode,Intuition,Time Steps in First Run,Path Length in First Run,Time Steps in Second Run,Path Length in Second Run,Coverage,Score,Computation Time,Maze Size,Number of Grid Visited Per Time Step of First Run,Number of Grid Visited Per Path Length of First Run
0,1,complete,False,8.43,13.07,0.00,0.00,0.00,0.28,8.44,0.0,0.03,0.03
1,1,complete,True,10.14,17.01,0.00,0.00,0.00,0.34,3.29,0.0,0.04,0.04
2,1,incomplete,False,33.31,40.84,1.23,1.28,0.18,1.71,15.69,0.0,0.03,0.04
3,1,incomplete,True,42.09,54.19,1.09,1.02,0.19,1.22,12.67,0.0,0.04,0.04
4,2,complete,False,14.14,24.27,0.00,0.00,0.00,0.47,11.01,0.0,0.03,0.04
5,2,complete,True,15.01,24.53,0.00,0.00,0.00,0.50,10.25,0.0,0.03,0.04
6,2,incomplete,False,50.33,61.62,1.05,0.45,0.19,1.51,39.09,0.0,0.03,0.03
7,2,incomplete,True,52.26,67.91,1.79,0.73,0.18,1.84,22.41,0.0,0.03,0.04
8,3,complete,False,15.91,28.60,0.00,0.00,0.00,0.53,24.57,0.0,0.03,0.04
9,3,complete,True,18.05,37.74,0.00,0.00,0.00,0.60,10.71,0.0,0.04,0.05


In [27]:
summary_mean.to_csv('summary_mean.csv', sep = ',', index = False)
summary_std.to_csv('summary_std.csv', sep = ',', index = False)